In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
kickstarter = pd.read_csv('kickstarter_cleaned_index.csv')
kickstarter_cleaned = kickstarter.drop(columns='Unnamed: 0')

print(kickstarter_cleaned.state.value_counts())

0            158968
1            133851
canceled      26744
live           2250
suspended      1230
Name: state, dtype: int64


Extremely Successful Kickstarters

In [18]:
kickstarter_cleaned['percent_funded'] = np.round((kickstarter_cleaned['usd_pledged_real'] / kickstarter_cleaned['usd_goal_real']),3)
kickstarter_cleaned['percent_funded'].describe()

np.percentile(kickstarter_cleaned['percent_funded'],99)

12.496

The median is 0.275, 50% of the projects get 0.275 times their goal -- failed. The 99th percentile of projects with highest percentage funded relative to their goal is 12.496. Let's discover these projects that have really high funding!

In [19]:
highly_funded = kickstarter_cleaned['percent_funded'] > 12.496

highly_funded_kickstarters = kickstarter_cleaned[highly_funded].sort_values(by='percent_funded',ascending=False)

highly_funded_kickstarters.head()

,ID,name,category,main_category,currency,deadline,goal,launched,state,backers,country,usd_pledged_real,usd_goal_real,project_length,pledged_per_backer,percent_funded
314975,9509582,VULFPECK /// The Beautiful Game,Music,Music,USD,2016-10-17 00:00:00,1.0,2016-08-18 09:04:03,1,3917,US,104277.89,1.0,59,26.621876,104277.89
158899,1947298033,Re-covering with Friends,Rock,Music,USD,2016-12-13 00:00:00,1.0,2016-10-14 19:04:27,1,955,US,68764.10,1.0,59,72.004293,68764.10
307740,907870443,VULFPECK /// Thrill of the Arts,Music,Music,USD,2015-10-09 00:00:00,1.0,2015-08-10 19:31:56,1,1673,US,55266.57,1.0,59,33.034411,55266.57
65151,1388400809,Energy Hook,Video Games,Games,USD,2013-06-10 00:00:00,1.0,2013-05-10 01:22:38,1,1622,US,41535.01,1.0,30,25.607281,41535.01
69465,1413857335,Band of Brothers 2nd Chance,Tabletop Games,Games,USD,2016-08-02 00:00:00,1.0,2016-07-12 00:29:12,1,268,US,32843.00,1.0,20,122.548507,32843.00


In [21]:
highly_funded_kickstarters['usd_goal_real'].value_counts(sort=True).head(5)

1.0        228
1000.0     193
10000.0    164
500.0      159
5000.0     152
Name: usd_goal_real, dtype: int64

In [22]:
highly_funded_kickstarters.main_category.value_counts()

Games           971
Design          752
Technology      446
Art             181
Comics          153
Fashion         143
Music           136
Publishing      133
Food             95
Film & Video     91
Crafts           91
Photography      22
Theater          10
Journalism        4
Dance             2
Name: main_category, dtype: int64

We found that there are many projects that have outlier success because their goals are too low. We take these projects out to see whether there is anything interesting with normal projects that do not have goal less than 50 dollars. 

In [26]:
kickstarter_cleaned.groupby('main_category')['percent_funded'].mean()

main_category
Art             2.906110
Comics          6.724753
Crafts          3.013387
Dance           0.870736
Design          2.219663
Fashion         1.247900
Film & Video    1.790400
Food            1.008408
Games           8.052188
Journalism      0.671604
Music           9.477993
Photography     0.751017
Publishing      2.231004
Technology      3.551625
Theater         1.098151
Name: percent_funded, dtype: float64

In [27]:
#take out goals that are too small

reasonable_goal = (kickstarter_cleaned['percent_funded'] > 12.496) & (kickstarter_cleaned['goal'] >= 25)

new_highly_funded = kickstarter_cleaned[reasonable_goal]
new_highly_funded.sort_values(by='percent_funded',ascending=False).head()

#newExtremeFunded.groupby('main_category')

,ID,name,category,main_category,currency,deadline,goal,launched,state,backers,country,usd_pledged_real,usd_goal_real,project_length,pledged_per_backer,percent_funded
49910,1296661668,Solid Watch,Product Design,Design,USD,2011-08-29 00:00:00,100.0,2011-05-31 07:01:49,1,474,US,134090.00,100.0,89,282.890295,1340.900
29845,1176482851,TREVOR HALL presents... The Fruitful Darkness,Music,Music,USD,2017-09-06 00:00:00,108.0,2017-08-07 20:17:39,1,2254,US,137570.93,108.0,29,61.034130,1273.805
138599,1825421712,Keyflower is Awesome!,Tabletop Games,Games,USD,2015-08-11 00:00:00,100.0,2015-06-26 19:51:07,1,2041,US,121033.00,100.0,45,59.300833,1210.330
17022,1100872151,The Let's Make a Bunch of Logos Project Part 3,Graphic Design,Design,USD,2012-01-14 00:00:00,25.0,2011-12-15 18:37:21,1,475,US,26243.58,25.0,29,55.249642,1049.743
160187,1955357092,Exploding Kittens,Tabletop Games,Games,USD,2015-02-20 00:00:00,10000.0,2015-01-20 19:00:19,1,219382,US,8782571.99,10000.0,30,40.033239,878.257


In [28]:
new_highly_funded.main_category.value_counts()

Games           885
Design          730
Technology      418
Fashion         129
Art             112
Comics          105
Publishing       88
Crafts           77
Food             70
Music            42
Film & Video     39
Photography      17
Journalism        3
Theater           2
Dance             1
Name: main_category, dtype: int64

In [30]:
new_highly_funded.pledged_per_backer.mean()

110.11446332865854

In [31]:
kickstarter_cleaned.pledged_per_backer.mean()

75.69599754197375

In [ ]:
extremelyFundedKickstarters[extremelyFundedKickstarters['usd_goal_real'] <= 120000]['usd_goal_real'].value_counts().sort_index().plot()

In [ ]:
extremelyFundedKickstarters['usd_goal_real'].describe()

In [ ]:
extremelyFundedKickstarters[extremelyFundedKickstarters['usd_goal_real'] <= 10000]['usd_goal_real'].describe()

In [ ]:
extremelyFundedKickstarters.groupby('main_category')['percent_funded'].describe()

Average Pledge